In [3]:
import tensorflow as tf
import numpy as np
import os
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, Embedding
from string import punctuation

In [4]:
sequence_length = 100
BATCH_SIZE = 128
EPOCHS = 30
# dataset file path
FILE_PATH = "data/1-tom.txt"
BASENAME = os.path.basename(FILE_PATH)
text = open(FILE_PATH, encoding="utf-8").read()
text = open(FILE_PATH, encoding="utf-8").read()
text = text.lower()
text = text.translate(str.maketrans("", "", punctuation))

In [5]:
# print some stats
n_chars = len(text)
vocab = ''.join(sorted(set(text)))
print("unique_chars:", vocab)
n_unique_chars = len(vocab)
print("Number of characters:", n_chars)
print("Number of unique characters:", n_unique_chars)

unique_chars: 
 01234567e«»абвгдежзийклмнопрстуфхчшщыьэюяіғқңүұһәө–﻿
Number of characters: 717547
Number of unique characters: 54


In [6]:
# dictionary that converts characters to integers
char2int = {c: i for i, c in enumerate(vocab)}
# dictionary that converts integers to characters
int2char = {i: c for i, c in enumerate(vocab)}

In [7]:
# save these dictionaries for later generation
pickle.dump(char2int, open(f"{BASENAME}-char2int.pickle", "wb"))
pickle.dump(int2char, open(f"{BASENAME}-int2char.pickle", "wb"))

In [8]:
# convert all text into integers
encoded_text = np.array([char2int[c] for c in text])
# construct tf.data.Dataset object
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
# print first 5 characters
for char in char_dataset.take(8):
    print(char.numpy(), int2char[char.numpy()])

53 ﻿
0 

0 

1  
1  
1  
1  
1  


In [9]:
sequences = char_dataset.batch(2*sequence_length + 1, drop_remainder=True)

for sequence in sequences.take(2):
    print(''.join([int2char[i] for i in sequence.numpy()]))

﻿

     қайтқанда


     1

үш күндік жолдың бүгінгі соңғы күніне шәкірт бала барын салды
қорықтан күн шыға атқа мінейік деп асыққанды бұны қаладан алып қайтқалы барған ағайыны байтасты да таң атаратпа
стан өзі оятып тұрғызып еді
күнұзын аттан да түспей өзге жүргіншілерден оқ бойы алда отырған кейкейде өзіне таныс көкүйрім мен буратиген тақырбұлақ сияқты қонысқұдықтардың тұстұсына келгенде бала оқшау


In [10]:
def split_sample(sample):
    ds = tf.data.Dataset.from_tensors((sample[:sequence_length], sample[sequence_length]))
    for i in range(1, (len(sample)-1) // 2):
        input_ = sample[i: i+sequence_length]
        target = sample[i+sequence_length]
        # extend the dataset with these samples by concatenate() method
        other_ds = tf.data.Dataset.from_tensors((input_, target))
        ds = ds.concatenate(other_ds)
    return ds

dataset = sequences.flat_map(split_sample)

In [11]:
def one_hot_samples(input_, target):
    # onehot encode the inputs and the targets
    return tf.one_hot(input_, n_unique_chars), tf.one_hot(target, n_unique_chars)

dataset = dataset.map(one_hot_samples)

In [12]:
# print first 2 samples
for element in dataset.take(2):
    print("Input:", ''.join([int2char[np.argmax(char_vector)] for char_vector in element[0].numpy()]))
    print("Target:", int2char[np.argmax(element[1].numpy())])
    print("Input shape:", element[0].shape)
    print("Target shape:", element[1].shape)
    print("="*50, "\n")

Input: ﻿

     қайтқанда


     1

үш күндік жолдың бүгінгі соңғы күніне шәкірт бала барын салды
қорықтан к
Target: ү
Input shape: (100, 54)
Target shape: (54,)

Input: 

     қайтқанда


     1

үш күндік жолдың бүгінгі соңғы күніне шәкірт бала барын салды
қорықтан кү
Target: н
Input shape: (100, 54)
Target shape: (54,)



In [13]:
ds = dataset.repeat().shuffle(1024).batch(BATCH_SIZE, drop_remainder=True)

In [14]:
model = Sequential([
    LSTM(256, input_shape=(sequence_length, n_unique_chars), return_sequences=True),
    Dropout(0.25),
    LSTM(256),
    Dense(n_unique_chars, activation="softmax"),
])

In [15]:
model_weights_path = f"results/{BASENAME}-{sequence_length}.h5"
model.summary()
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 256)          318464    
                                                                 
 dropout (Dropout)           (None, 100, 256)          0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 54)                13878     
                                                                 
Total params: 857,654
Trainable params: 857,654
Non-trainable params: 0
_________________________________________________________________


In [16]:
# make results folder if does not exist yet
if not os.path.isdir("results"):
    os.mkdir("results")
model.fit(ds, steps_per_epoch=(len(encoded_text) - sequence_length) // BATCH_SIZE, epochs=EPOCHS)
model.save(model_weights_path)

Epoch 1/30
5605/5605 [==============================] - 187s 32ms/step - loss: 1.8853 - accuracy: 0.4065
Epoch 2/30
5605/5605 [==============================] - 183s 33ms/step - loss: 1.5553 - accuracy: 0.5026
Epoch 3/30
5605/5605 [==============================] - 184s 33ms/step - loss: 1.4470 - accuracy: 0.5354
Epoch 4/30
5605/5605 [==============================] - 183s 33ms/step - loss: 1.3749 - accuracy: 0.5558
Epoch 5/30
5605/5605 [==============================] - 184s 33ms/step - loss: 1.3194 - accuracy: 0.5721
Epoch 6/30
5605/5605 [==============================] - 183s 33ms/step - loss: 1.2738 - accuracy: 0.5859
Epoch 7/30
5605/5605 [==============================] - 183s 33ms/step - loss: 1.2354 - accuracy: 0.5967
Epoch 8/30
5605/5605 [==============================] - 183s 33ms/step - loss: 1.2027 - accuracy: 0.6067
Epoch 9/30
5605/5605 [==============================] - 184s 33ms/step - loss: 1.1748 - accuracy: 0.6151
Epoch 10/30
5605/5605 [==============================] 

**Generating Text**

In [30]:
import numpy as np
import pickle
import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Activation
import os

sequence_length = 100
# dataset file path
FILE_PATH = "data/1-tom.txt"
BASENAME = os.path.basename(FILE_PATH)

seed = "алыста келе жатқан жолаушыны көріп"

# load vocab dictionaries
char2int = pickle.load(open(f"{BASENAME}-char2int.pickle", "rb"))
int2char = pickle.load(open(f"{BASENAME}-int2char.pickle", "rb"))
vocab_size = len(char2int)

model = Sequential([
    LSTM(256, input_shape=(sequence_length, vocab_size), return_sequences=True),
    Dropout(0.25),
    LSTM(256),
    Dense(vocab_size, activation="softmax"),
])

model.load_weights(f"results/{BASENAME}-{sequence_length}.h5")

s = seed
n_chars = 400
generated = ""
for i in tqdm.tqdm(range(n_chars), "Generating text"):
    # make the input sequence
    X = np.zeros((1, sequence_length, vocab_size))
    for t, char in enumerate(seed):
      X[0, (sequence_length - len(seed)) + t, char2int[char]] = 1
    # predict the next character
    predicted = model.predict(X, verbose=0)[0]
    # converting the vector to an integer
    next_index = np.argmax(predicted)
    # converting the integer to a character
    next_char = int2char[next_index]
    # add the character to results
    generated += next_char
    # shift seed and the predicted character
    seed = seed[1:] + next_char

print("Seed:", s)
print("Generated text:")
print(generated)

Generating text: 100%|██████████| 400/400 [00:23<00:00, 17.31it/s]

Seed: алыста келе жатқан жолаушыны көріп
Generated text:
 екеуінің де бірі бала да алшынбай бұл ауылда бар атшабарлар абайдың жаңағы ашуы бар басты байсалды байқаса қарап алып бұйрық етті
жақып таныс алған жоқ
бірақ құнанбай аулынан қарап тұрып барып абай байдалының арасынан басқан екен абай жаңағы «тобықтығы» деген сөздер болатын
осы кезде абай жаңағы «бұл  деді
содан бері бұға бұған соқ теңіздеп айтқан жақында абайға қарап алып еді үй іші тегіс жатыр 
